In [1]:
import urllib.request 
import urllib.parse
import urllib.error
import ssl
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup as BS
import re
import string
import pandas as pd
import csv
import time

In [2]:
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [11]:
property_url=[]


urls = ['https://www.redfin.com/county/303/CA/Alameda','https://www.redfin.com/county/303/CA/Alameda-County','https://www.redfin.com/county/303/CA/Alameda-County/page-2',
        'https://www.redfin.com/county/303/CA/Alameda-County/page-3','https://www.redfin.com/city/13654/CA/Oakland',
        'https://www.redfin.com/city/13654/CA/Oakland/page-2','https://www.redfin.com/city/14986/CA/Pleasanton',
        'https://www.redfin.com/city/17151/CA/San-Francisco','https://www.redfin.com/city/17420/CA/San-Jose','https://www.redfin.com/city/17420/CA/San-Jose/page-2',
        'https://www.redfin.com/city/17151/CA/San-Francisco/page-2','https://www.redfin.com/city/17151/CA/San-Francisco/page-3','https://www.redfin.com/city/17151/CA/San-Francisco/page-4','https://www.redfin.com/city/17151/CA/San-Francisco/page-5','https://www.redfin.com/city/17151/CA/San-Francisco/page-6',
        'https://www.redfin.com/city/12204/CA/Milpitas','https://www.redfin.com/county/303/CA/Alameda-County',
        'https://www.redfin.com/city/17447/CA/San-Leandro','https://www.redfin.com/county/321/CA/Los-Angeles-County',
       'https://www.redfin.com/city/8439/CA/Hayward','https://www.redfin.com/city/8439/CA/Hayward/page-2','https://www.redfin.com/county/339/CA/San-Diego-County']
for url in urls:
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()
    soup = BS(webpage,'html.parser')
    url=soup.find_all('a',{'data-rf-test-name':'basic-card-photo'})
    # print(url)
    for href in url:
      href = "https://www.redfin.com/"+str(href.get('href'))
      property_url.append(href)
    print(len(property_url))

41
82
123
164
205
246
279
320
361
402
443
484
525
566
607
648
689
730
771
812
853
894


In [12]:
sample_url='https://www.redfin.com//CA/Alameda/2260-San-Jose-Ave-94501/unit-A/home/1746620'

In [5]:
house_req = Request(sample_url, headers={'User-Agent': 'Mozilla/5.0'})
house_webpage = urlopen(house_req).read()
attri_soup = BS(house_webpage,'html.parser')
# print(attri_soup)
street = attri_soup.find("div",{"class":"street-address"}).text.replace(',','')
city_state=attri_soup.find('div',{'data-rf-test-id':'abp-cityStateZip'}).text
walkscore = attri_soup.find('div',{'class':'transport-icon-and-percentage walkscore'}).text
transitscore = attri_soup.find('div',{'class':'transport-icon-and-percentage transitscore'}).text
bikescore = attri_soup.find('div',{'class':'transport-icon-and-percentage bikescore'}).text
schoolrating = attri_soup.find_all('span',{'class':'rating-num'})
competitivescore = attri_soup.find('div',{'class':'scoreTM'}).text
price=float(attri_soup.find('div',{'class':'statsValue'}).text.replace('$','').replace(',',''))
bed=attri_soup.find_all('div',{'class':'stat-block beds-section'})[1].text
bath=attri_soup.find('div',{'class':'stat-block baths-section'}).text
sqft=attri_soup.find('div',{'class':'stat-block sqft-section'}).text
# details=attri_soup.find_all('div',{'class':"keyDetailsList"})
details=attri_soup.find_all('div',{'class':"keyDetail font-weight-roman font-size-base"})
print(walkscore, transitscore, bikescore)
print(competitivescore, price, sqft, bed, bath)
print(street,city_state)
# print(details)

91 / 100 49 / 100 74 / 100
87Very CompetitiveRedfin Compete Score™ 689000.0 572Sq Ft 1Bed 1Bath
2260 A San Jose Ave Alameda, CA 94501


In [6]:
i=1
dic1={'street':[],'city_state':[],'price':[],'bed':[],'bath':[],'sqft':[],'walkscore':[],'transitscore':[],'bikescore':[],
    'competitivescore':[],'url':[]}
for sample_url in property_url:
    house_req = Request(sample_url, headers={'User-Agent': 'Mozilla/5.0'})
    house_webpage = urlopen(house_req).read()
    attri_soup = BS(house_webpage,'html.parser')
    # print(attri_soup)
    street = attri_soup.find("div",{"class":"street-address"}).text.replace(',','')
    city_state=attri_soup.find('div',{'data-rf-test-id':'abp-cityStateZip'}).text
    
    price=attri_soup.find('div',{'class':'statsValue'}).text.replace('$','').replace(',','')
    try:
        bed=attri_soup.find_all('div',{'class':'stat-block beds-section'})[1].text
    except:
        bed = 0
    try:
        bath=attri_soup.find('div',{'class':'stat-block baths-section'}).text
    except:
        bath = 0
    try:
        sqft=attri_soup.find('div',{'class':'stat-block sqft-section'}).text
    except:
        sqft = 0
    try:
        walkscore = attri_soup.find('div',{'class':'transport-icon-and-percentage walkscore'}).text
    except:
        walkscore = 0
    try:
        transitscore = attri_soup.find('div',{'class':'transport-icon-and-percentage transitscore'}).text
    except:
        transitscore = 0    
    try:
        bikescore = attri_soup.find('div',{'class':'transport-icon-and-percentage bikescore'}).text
    except:
        bikescore = 0
    try:
        competitivescore = attri_soup.find('div',{'class':'scoreTM'}).text
    except:
        competitivescore = 0
   # print(i, sample_url)
   # print(price, bed, bath, sqft)
   # print(street,city_state, walkscore, transitscore, bikescore, competitivescore)
    
    #adding data to dictionary
    dic1['street'].append(street)
    dic1['city_state'].append(city_state)
   # dic['state'].append(state)
   # dic['zipcode'].append(zipcode)
    dic1['price'].append(price)
   # dic['mortgage'].append(mortgage)
    dic1['bed'].append(bed)
    dic1['bath'].append(bath)
    dic1['sqft'].append(sqft)
    dic1['walkscore'].append(walkscore)
    dic1['transitscore'].append(transitscore)
    dic1['bikescore'].append(bikescore)
    dic1['competitivescore'].append(competitivescore)
  #  dic['ac'].append(ac)
   # dic['hoa'].append(hoa)
    dic1['url'].append(sample_url)
  #  dic['price_per_sqft'].append(price_per_sqft)
    print(i)
    i+=1

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41


In [40]:
df=pd.DataFrame.from_dict(dic1, orient='index') #convert dictionary to dataframe and add Nan to unfilled values
df=df.T #transpose dataframe
df
df.to_csv('scrape_data_v1.csv') #Convert dataframe to csv file

In [20]:
import pandas as pd
import numpy as np
df=pd.read_csv('scrape_data_v1.csv',index_col=0)

df.dropna(axis=0,subset=['street', 'city', 'state', 'zipcode', 'price', 'sqft',
       'walkscore', 'transitscore', 'bikescore', 'competitivescore',                  #drop nulls because some entries have only unique records which inappropriately biases the model
       'price_per_sqft','bed'],inplace=True)

df.drop(df[df['price_per_sqft']=='#VALUE!']['price_per_sqft'].index,inplace=True)
df.drop(df[df['sqft']=="Äî'"]['sqft'].index,inplace=True)

df['bed']=df[['bed']].applymap(lambda x: x.encode('utf-8').decode('ascii', 'ignore')) # coverting special charatcters to blanks
df['bath']=df[['bath']].applymap(lambda x: x.encode('utf-8').decode('ascii', 'ignore'))

df['bed'].replace('', np.nan, inplace=True) #converting blanks to nan
df.dropna(subset=['bed'], inplace=True)     #dropping nan values
df['bath'].replace('', np.nan, inplace=True)
df.dropna(subset=['bath'], inplace=True)

df.drop('url',axis=1,inplace=True) #dropping url column


df['city']=df['city'].apply(lambda x:x[0]+x[1:].lower()) #normalizing values across all cities


df['walkscore']=df['walkscore'].astype(int) # converting walkscore to integer

df['price']=df['price'].astype(int) # converting price to integer
df.head()

len(df)
df.to_csv('scrape_data_v2.csv') #Save to V2 file
